# Classification with EfficientNetV2

* Original Google Repo: https://github.com/google/automl/tree/master/efficientnetv2
* Paper published 2021

In [ ]:
import math, re, os
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
print(tf.__version__)
print(tfa.__version__)

from flowerclass_read_tf_ds import get_datasets
import tensorflow_hub as hub
import pandas as pd
import math
import plotly_express as px

In [ ]:
tf.test.gpu_device_name()

# I. Data Loading

* Choose 480x480 as model is fixed: https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_l/feature_vector/2

In [ ]:
image_size = 224
batch_size = 64

In [ ]:
#%%debug (50, 480)
ds_train, ds_valid, ds_test = get_datasets(BATCH_SIZE=batch_size, IMAGE_SIZE=(image_size, image_size), 
                                           RESIZE=None, tpu=False)

# II. Model Setup: EfficientNetV2

In [ ]:
#effnet2_base = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_l/feature_vector/2"
#effnet2_base = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_m/feature_vector/2"
effnet2_base = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2"

In [ ]:
hub.KerasLayer

In [ ]:
    
effnet2_tfhub = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=(image_size, image_size,3)),
    hub.KerasLayer(effnet2_base, trainable=False),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(104, activation='softmax')
])
effnet2_tfhub.build((None, image_size, image_size,3,)) #This is to be used for subclassed models, which do not know at instantiation time what their inputs look like.


effnet2_tfhub.summary()

Notice large amounts of untrainable params as efficientnetv2 layers are frozen

In [ ]:
effnet2_tfhub.layers

In [ ]:
layer = effnet2_tfhub.layers[0]
print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

In [ ]:
layer.weights[0].shape


In [ ]:
layer.trainable

Why?

# III. Training

Keras Transfer Learning: https://keras.io/guides/transfer_learning/

# IIIa) Phase I: Train Top Layer (frozen layers)

### Optimize Training for Compute Infrastructure

In [ ]:
effnet2_tfhub.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=[tfa.metrics.F1Score(num_classes=104, average='macro'), tf.keras.metrics.CategoricalAccuracy(
    name='categorical_accuracy', dtype=None)])

* batchsize:4 with 512 resized to 480px OOM

* `effnet2L_tfhub.fit(ds_train, epochs=1, validation_data=ds_valid, batch_size=batch_size, steps_per_epoch=1)`

#### EfficientNetV2 Large

* try batchsize 4, 8, 16 and image size  224, 331 (without resizing for now)
* bs/image size (no resize)
    * 8/224: pass
    * 16/224 pass
    * 32/224 pass
    * 64/224 pass
    * 128/224 pass
* try 331 (second largest size of images available) with efficientetV2 small
    * 16/331: OOM
    * 8/331: OOM

* Test with optimal 480x480 input:
    
   * 8/448 (resized 480): OOM
   * 8/224 (resized 480): OOM
   * 2/224 (resized 480): OOM
   > Resizing to the optimal 480x480 image size not possible with EfficientNetV2 Large due to OOM

    
    
#### EfficientNetV2 Medium

* Test with optimal 480x480 input:

   * 2/224 (resized 480): OOM

#### EfficientNetV2 Small

* Test with optimal 384 x 384: OOM

> All 3 model types, small, medium, large cannot be used with their optimal resolution.
> 




In [ ]:
compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / batch_size))
steps_per_epoch_tr = compute_steps_per_epoch(12753)
steps_per_epoch_val = compute_steps_per_epoch(3712)
steps_per_epoch_tr, steps_per_epoch_val

In [ ]:
callback_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1_score', min_delta=0, patience=5, verbose=1,
    mode='max', baseline=None, restore_best_weights=False
)
callback_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="training/cp-{epoch:04d}.ckpt",
                                                 save_weights_only=True,
                                                               monitor='val_f1_score',
                                                 verbose=1,  mode='max', save_best_only=True)

history = effnet2_tfhub.fit(ds_train, epochs=40, validation_data=ds_valid, 
                            batch_size=batch_size, 
                            steps_per_epoch= steps_per_epoch_tr,
                            validation_steps=steps_per_epoch_val,
                           callbacks=[callback_stopping, callback_model_checkpoint], shuffle=True)

In [ ]:
effnet2_tfhub.save('saved_model/my_model_phase1')

In [ ]:
results_tr = pd.DataFrame.from_dict(history.history)
results_tr['epochs'] = results_tr.index + 1
results_tr.head()

results_to_plot = results_tr.melt(id_vars="epochs")
results_to_plot.head()

In [ ]:
results_to_plot['variable'].unique()

In [ ]:
px.line(data_frame=results_to_plot[results_to_plot.variable.isin(['loss', 'val_loss'])],
           x='epochs', y='value', color="variable")

In [ ]:
px.line(data_frame=results_to_plot[results_to_plot.variable.isin(['f1_score', 'val_f1_score'])],
           x='epochs', y='value', color="variable")

In [ ]:
best_phase1_f1 = results_tr['val_f1_score'].max()
best_phase1_epoch = results_tr.loc[results_tr['val_f1_score'] == best_phase1_f1, 'epochs'].values[0]


In [ ]:
best_phase1_f1, best_phase1_epoch

## IIIb) Phase II: Unfreeze and FineTuning

Unfreeze weights, try fine tuning whole network

In [ ]:
effnet2_tfhub.trainable = True

In [ ]:
effnet2_tfhub.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=[tfa.metrics.F1Score(num_classes=104, average='macro'), tf.keras.metrics.CategoricalAccuracy(
    name='categorical_accuracy', dtype=None)])

In [ ]:
callback_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1_score', min_delta=0, patience=5, verbose=1,
    mode='max', baseline=None, restore_best_weights=False
)
callback_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="training2/cp-{epoch:04d}.ckpt",
                                                 save_weights_only=True,
                                                               monitor='val_f1_score',
                                                 verbose=1, mode='max', save_best_only=True)

history = effnet2_tfhub.fit(ds_train, epochs=10, validation_data=ds_valid, 
                            batch_size=batch_size, 
                            steps_per_epoch=steps_per_epoch_tr,
                            validation_steps=steps_per_epoch_val,
                           callbacks=[callback_stopping, callback_model_checkpoint], shuffle=True)

In [ ]:
effnet2_tfhub.save('saved_model/my_model_phase2')

In [ ]:
results_tr = pd.DataFrame.from_dict(history.history)
results_tr['epochs'] = results_tr.index + 1
results_tr.head()

results_to_plot = results_tr.melt(id_vars="epochs")
results_to_plot.head()

In [ ]:
px.line(data_frame=results_to_plot[results_to_plot.variable.isin(['loss', 'val_loss'])],
           x='epochs', y='value', color="variable")

In [ ]:
px.line(data_frame=results_to_plot[results_to_plot.variable.isin(['f1_score', 'val_f1_score'])],
           x='epochs', y='value', color="variable")

### Load best model, either phase 1 or 2

In [ ]:
best_phase2_f1 = results_tr['val_f1_score'].max()

if best_phase1_f1 > best_phase2_f1:
    effnet2_tfhub.load_weights("training/"+"cp-"+f"{best_phase1_epoch}".rjust(4, '0')+".ckpt")
    print(f"best phase 1: {best_phase1_f1}")
else:
    print(f"best phase 2: {best_phase2_f1}")



# IV. Submission

id,label
a762df180,0
24c5cf439,0
7581e896d,0
eb4b03b29,0
etc.

In [ ]:
test_pred = effnet2_tfhub.predict(ds_test, batch_size=batch_size)


In [ ]:
img_ids = []
img_preds = []
for imgs, idnum in ds_test:
    img_preds.append(effnet2_tfhub.predict(imgs, batch_size=batch_size))
    img_ids.append(idnum)

In [ ]:
img_ids = np.concatenate([img_id.numpy() for img_id in img_ids])


In [ ]:
img_preds = np.concatenate([img_pred.argmax(1) for img_pred in img_preds])

In [ ]:
img_ids.shape, img_preds.shape

In [ ]:
submission = pd.DataFrame({"id": img_ids, "label": img_preds})
submission['id'] = submission['id'].apply(lambda x: x.decode())

In [ ]:
submission.head()

In [ ]:
submission.dtypes

In [ ]:
submission.to_csv("submission.csv", index=False)